---
title: "Modeling Heat Flow - Part 3"
description: "Modeling the sun"
author: "Steven Wolf"
date: "12/13/2024"
number-sections: false
categories:
  - Heat Equation
  - Astrophysics
  - Python
  - Modeling
  - 3D system
  - Finite Difference Method
  - Animating Plots
execute: 
  messages: false
  warning: false
jupyter: python3
draft:
  true
---


## Overview

I've been in the weeds on this for a while, so it's time to zoom back to the big picture. Specifically, I'm working on a model of heat flow in my house because it gets hot in parts of my house during the summer even with the AC going full blast.  I've written a series of posts [introducing the heat equation](../heatEqn1/modelHeatFlow.qmd), [modeling solar heat generation](../heatEqn2/heatEqn2.qmd), and [visualizing the dynamics of a simple system](../heatEqn2/hotBoxViz.qmd). However, the limiting approximation being made was that the sun's rays were directly incident on the top surface **at all times**. This made the calculation easy, but is clearly not realistic.  This is going to be a little mathy, but I can make some pictures to support the mathematics.

## Locating the sun

The location of the sun is important because the power deposited into a surface from any light source is related to the angle of incidence. Consider the figure below:

![](surface.png){fig-align="center" width=400} 

The orange ray indicates the direction of the sunlight at the surface, and the dotted line indicates the direction of the line that is perpendicular to the surface. The power generated in the surface due to sunlight will be:
$$
P = I_{\text{Sun}} A \cos^2\beta
$$
where $I_\text{sun}$ is the intensity of the sunlight and $A$ is the area of the surface.  This assumes things like the size of the surface is much smaller than the distance from that surface to the sun. But this angle depends on the following:

- The orientation of the surface relative to the surface of the earth. For example is it horizontal or vertical? And if it is vertical (like a wall) is it facing north or east?
- The time of day. As we learn in grade school, the sun rises in the east and sets in the west.
- The latitude of the surface.
- The time of year.

Given all of these things, it is not as simple as stating that the sun starts at $\beta=-90\deg$ at sunrise and sets at $\beta=+90\deg$.  So I'm going to develop a framework for calculating this.

### Terrestrial coordinate system
I'm going to develop a terrestrial coordinate system so that I can calculate the angle of incidence.  I'll begin by considering the tilt of the earth's rotation axis relative to the direction of sunlight. In the figures below, sunlight is in the direction of the yellow rays, the earth is at the origin of the coordinate system, and the z direction is the earth's rotation axis. (I'm indicating the poles by the white blobs on the sphere, and the north pole has the red arrow representing the +z axis poking through it).

![](seasons.png){fig-align="center" width=800} 

We know that seasons change because the sun's ray's are more/less direct at different latitudes, and this changes because the angle of the earth's rotation axis changes relative to the direction of sunlight.  I've labeled this angle $\alpha$ in the figure above. We know that it varies from $+23.5 \deg$ to $-23.5 \deg$ on the winter solstice and the summer solstice respectively. On the equinoxes, $\alpha=0 \deg$ as depicted above.  We can write a function for $\alpha$ as follows:

$$
\alpha(T) = \alpha_0 \cos(\Omega_y T)
$$
where $T$ is the number of days after the winter solstice, and $\Omega_y = \frac{2\pi}{1\,\text{yr}}$ is the angular frequency of 1 year. Below, I will create a function `axisTilt` for calculating this angle based on the fraction of a year that has passed since the winter solstice.


In [ ]:
import numpy as np

def axisTilt(T):
  maxTilt = 23.5
  omegaYear = 2*np.pi
  axisTilt = maxTilt * np.cos(omegaYear*T)

Since we need to account for changing $\alpha$, I will need to include as a variable, this is done on the figure below. I will now use this to determine the direction of the sun's rays $\hat{R}$ in terms of the earth's coordinate system. To aid in this, I will choose the $xz$ plane to be in the plane of the drawing, and the sun's rays will be along the unit vectors in the $x$ and $z$ directions as follows:

$$
\hat{R} = \cos\alpha \hat{i} + \sin\alpha \hat{k}
$$

![](earthViz.png){fig-align="center" width=600} 

If we consider a point on the surface of the earth, it will be useful to use spherical coordinates.  The unit vectors in the $\hat{r}$ and $\hat{\theta}$ directions are depicted on the drawing above ($\hat{\phi}$ is into the screen at the location given).  We know that the spherical unit vectors are:

$$
\begin{align*}
  \hat{r} &= \sin\theta\cos\phi \hat{i} + \sin\theta\sin\phi \hat{j} + \cos\theta \hat{k} \\
  \hat{\theta} &= \cos\theta\cos\phi \hat{i} + \cos\theta\sin\phi \hat{j} -\sin\theta\hat{k} \\
  \hat{\phi} &= -\sin\phi \hat{i} + \cos\phi\hat{j} \\
\end{align*}
$$

Using this coordinate system allows us to interpret several things. First, the polar angle is related to the angle of latitude.
$$
\theta = 90\deg - \theta_{L}
$$
where $\theta_L$ is the angle of Latitude at the location of interest (North latitude is positive, South latitude is negative.) Yes, I'm using degrees. Yes, it feels kind of wrong to use degrees. I like using the most natural units I can ($\hbar=c=1$ anyone?). I'll have to convert to radians when I code, but when I look up latitudes on the map, I get degrees.  As I tell my students, knowing what units you have is more important that putting everything into SI units. But I digress.  The relationship between the polar angle and the angle of latitude has the meaning that:
$$
\sin{\theta} = \cos\theta_L \quad \text{and} \cos\theta = \sin\theta_L
$$

Next, the time of day can be related to the azimuthal angle:
$$
\phi = \Omega t
$$
where $\Omega = \frac{2\pi}{1 \text{ day}}$ is the angular frequency of the earth's rotation. So $\phi=0$ implies $t=0$, which is solar midnight. (No need to standardize clocks across the planet for this.)

Finally, we can also state that the unit vectors noted above are related to the cardinal directions and the local vertical. That is $\hat{r}$ is the local vertical direction, $\hat{\theta}$ points South, and $\hat{\phi}$ points East.

### Day/night
Consider the dot product:
$$
\hat{R}\cdot\hat{r} = \cos\alpha\cos\theta_L\cos(\Omega t) + \sin\alpha\sin\theta_L
$$
If this product is positive, the point of interest is on the night-side of the planet, and if negative, the point of interest is on the day-side of the planet. If this product is 0, then that point is at sunrise or sunset. Let's turn this into a function to determine if it is daytime given the time of day, latitude, and axis tilt. I won't actually use this function, but this suggests a method of determining when a surface is exposed to sunlight.


In [ ]:
def isDaylight(t,axisTilt,thetaLatitude):
  oneDay = 24*60*60 # seconds
  Omega = 2*np.pi/oneDay
  alpha = axisTilt*np.pi/180
  thetaL = thetaLatitude*np.pi/180
  rDotR = np.cos(alpha)*np.cos(thetaL)*np.cos(Omega*t) + np.sin(alpha)*np.sin(thetaL)
  if rDotR > 0:
    isDay = False
  else:
    isDay = True
  
  return isDay

Note, that we can determine the sunrise/sunset times if we set the dot product above equal to zero and solve the following equation for $t$:
$$
\cos(\Omega t) = - \tan\alpha \tan\theta_L
$$
Implicitly, we are restricting $t$ to be less than one solar day.  Below, I will use this to determine the length of the summer and winter solstice days here in Greenville, NC (Latitude = $35.6 \deg$ N).


In [ ]:
#| code-fold: true

def dayCalc():
  oneDay = 24 # hours
  Omega = 2 * np.pi / oneDay
  alphaMax = 23.5*np.pi/180
  greenvilleLatitude = 35.6*np.pi/180
  
  winterSolTime = 24-2*np.arccos(-np.tan(+alphaMax)*np.tan(greenvilleLatitude))/Omega
  summerSolTime = 24-2*np.arccos(-np.tan(-alphaMax)*np.tan(greenvilleLatitude))/Omega
  print(f'The summer solstice has {summerSolTime:.1f} hours of sunlight in Greenville, NC.')
  print(f'The winter solstice has {winterSolTime:.1f} hours of sunlight in Greenville, NC.')
  
dayCalc()

### Temperature cycle
Now that we can put all of these together, I'm going to make decisions about modeling air temperature.  I will make some assumptions.

1. The daily low temperature occurs at sunrise, the daily high temperature occurs at sunset.
2. Temperature will smoothly increase (like a cosine) during the daylight and smoothly decrease (also like a cosine) during the nighttime.


In [ ]:
import matplotlib.pyplot as plt

def getSunriseSunset(axisTilt,thetaLatitude):
  oneDay = 24*60*60 # seconds
  Omega = 2*np.pi/oneDay
  alpha = axisTilt*np.pi/180
  thetaL = thetaLatitude*np.pi/180
  tSunrise = np.arccos(-np.tan(alpha)*np.tan(thetaL))/Omega
  tSunset = oneDay - tSunrise
  return [tSunrise, tSunset]

def airTemp(t,lowT, highT, axisTilt, thetaLatitude):
  oneDay = 24*60*60 # seconds
  avgTemp = (lowT+highT)/2
  tempDiff = (highT-lowT)/2
  tRise, tSet = getSunriseSunset(axisTilt,thetaLatitude)
  tDaylight = tSet-tRise
  OmegaDay = np.pi/tDaylight
  OmegaNight = np.pi/(oneDay-tDaylight)
  
  if(t < tRise):
    airTemp = avgTemp - tempDiff * np.cos(OmegaNight * (t-tRise))
  elif(t < tSet):
    airTemp = avgTemp - tempDiff * np.cos(OmegaDay * (t-tRise))
  else:
    airTemp = avgTemp + tempDiff * np.cos(OmegaNight * (t-tSet))
  
  return airTemp
  
def makeTempPlot(lowT, highT, axisTilt, thetaLatitude):
  oneDay = 24*60*60 # seconds
  t = np.linspace(0,oneDay,24*60)
  temp = np.zeros(len(t))
  for i in range(len(t)):
    temp[i] = airTemp(t[i],lowT, highT, axisTilt, thetaLatitude)
  T = getSunriseSunset(axisTilt,thetaLatitude)
  hrs = np.linspace(0,24,25).astype(int)
  plt.plot(t,temp)
  plt.axvline(T[0], linestyle='dashed',color='darkgrey')
  plt.axvline(T[1], linestyle='dashed',color='darkgrey')
  plt.xticks(ticks=np.linspace(0,oneDay,25),labels=hrs)
  plt.xticks(ticks=[],minor=True)
  plt.xlabel('Time of day (hr)')
  plt.ylabel('Temp (deg C)')
  plt.show()

makeTempPlot(22,30,-22,35.6)

## Returning to the box
Let's integrate all of this astrophysics into our box.

### Solar power generation on a box
So back to the simple box system, I can set up my box so that it has two faces in the N/S direction, and two faces in the E/W direction.  I will set things up much as I have done before:


In [ ]:
# Heat parameters
thermalDiffusivity = 22.39e-6 # meters^2/s for air
heatTransferCoef = 1 # For a typical metal to air W/m^2K
thermalConductivity = 50 # For a typical metal W/mK
specificHeat = 1000 # for aluminum J/kg K
wallDensity = 3000 # kg/m^3 for aluminum
solarIntensity = 1000 # W/m^2

## Results from previous optimization
optA = 1.117617e-03
optB = 6.666691e-03
solarThickness = 1/(specificHeat*wallDensity*optA) # m
conductionThickness = heatTransferCoef/(specificHeat*wallDensity*optB) 

# Length parameters (meters)
L = 2
W = 1
H = 1

Deltax = 0.05
xmax = int(L/Deltax)
ymax = int(W/Deltax)
zmax = int(H/Deltax)

xmid = xmax // 2
ymid = ymax // 2
zmid = zmax // 2

xgrid = np.linspace(0,L,xmax+1)
ygrid = np.linspace(0,W,ymax+1)
zgrid = np.linspace(0,H,zmax+1)

u0 = np.empty((xmax,ymax,zmax))

Now let's create an improved power generation function. The parameters are:

- `umat` The existing temperature matrix (in Celsius)
- `t` The current time (in seconds)
- `startDay` The number of days since the winter solstice that we are starting with
- `thetaLatitude` The latitude (in degrees) of the system
- `intensity` The intensity of sunlight at the earth's surface (about 1000 W/m^2 on a clear day)
- `c` The specific heat of the material making up the system's walls
- `rho` The density of the material making up the system's walls
- `penDepth` The effective penetration depth of sunlight into the surface


In [ ]:
def powerGen(t, umat, startDay, thetaLatitude, intensity, c, rho, penDepth):
  topFrac = 0
  northFrac = 0
  southFrac = 0
  eastFrac = 0
  westFrac = 0
  powerGen = np.zeros_like(umat)
  topPowerGen = powerGen
  northPowerGen = powerGen
  southPowerGen = powerGen
  eastPowerGen = powerGen
  westPowerGen = powerGen
  A = 1/(c*rho*penDepth)
  oneDay = 24*60*60 # seconds
  Omega = 2*np.pi/oneDay
  daysSinceWinterSolstice = startDay + t // oneDay
  tiltAngleRad = axisTilt(daysSinceWinterSolstice)*np.pi/180
  thetaL = thetaLatitude*np.pi/180
  rDotR = np.cos(tiltAngleRad)*np.cos(thetaL)*np.cos(Omega*t) + np.sin(tiltAngleRad)*np.sin(thetaL)
  # If it is daytime, need to add power, else, no power added
  if rDotR <=0:
    topPowerGen[:,:,-1].fill(A * rDotR**2 * intensity)
    # How much power to add to each wall?
    thetaDotR = np.cos(tiltAngleRad)*np.sin(thetaL)*np.cos(Omega*t) + np.sin(tiltAngleRad)*np.cos(thetaL)
    phiDotR = -np.cos(tiltAngleRad)*np.sin(Omega*t)
    # If southern exposure add to south wall, otherwise, there is northern exposure
    if thetaDotR <= 0: 
      southPowerGen[:,0,:].fill(A * thetaDotR**2 * intensity)
    else:
      northPowerGen[:,-1,:].fill(A * thetaDotR**2 * intensity)
    # If eastern exposure add to east wall, otherwise, there is western exposure
    if phiDotR <= 0: 
      eastPowerGen[0,:,:].fill(A * phiDotR**2 * intensity)
    else:
      westPowerGen[0,:,:].fill(A * phiDotR**2 * intensity)
  powerGen = topPowerGen + northPowerGen + southPowerGen + eastPowerGen + westPowerGen
  return powerGen

### Boundary convection

This can be integrated with the existing code with a few modifications. Specifically, in the boundary convection function, I will use the heat transfer coefficient, specific heat, and density of the system's walls, as well as the effective thickness of the convection boundary. I will also allow the ground to be a different temperature than the air


In [ ]:
#| code-fold: true

def bdryConv(t, umat, Tair, Tground, heatTransferCoef, c, rho, convThickness):
  Bair = heatTransferCoef/(c*rho*convThickness)
  Bground = Bair/100
  
  bdryTemp = np.zeros_like(umat)
  uSurf = np.zeros_like(umat)
  duConvdt = np.zeros_like(umat)

  bdryTemp[0,:,:].fill(Tair)
  bdryTemp[:,0,:].fill(Tair)
  bdryTemp[:,:,0].fill(Tground)
  bdryTemp[-1,:,:].fill(Tair)
  bdryTemp[:,-1,:].fill(Tair)
  bdryTemp[:,:,-1].fill(Tair)

  uSurf[0,:,:] = umat[0,:,:]
  uSurf[:,0,:] = umat[:,0,:]
  uSurf[:,:,0] = umat[:,:,0]
  uSurf[-1,:,:] = umat[-1,:,:]
  uSurf[:,-1,:] = umat[:,-1,:]
  uSurf[:,:,-1] = umat[:,:,-1]

  # Heat transfer with the air
  duConvdt[:,:,1:] = Bair*(bdryTemp[:,:,1:] - uSurf[:,:,1:])
  # Heat transfer with the ground
  duConvdt[:,:,0] = Bground*(bdryTemp[:,:,0] - uSurf[:,:,0])
  
  return duConvdt    

def lap3DFE(umat,dx):
    lap = np.empty_like(umat)

    # Interior elements:
    lap[1:-1,1:-1,1:-1] = (umat[:-2, 1:-1, 1:-1] + umat[2:, 1:-1, 1:-1] + umat[1:-1, :-2, 1:-1] + 
                           umat[1:-1, 2:, 1:-1] + umat[1:-1,1:-1,:-2] + umat[1:-1,1:-1,2:] - 6*umat[1:-1,1:-1,1:-1]) / dx**2

    # Surface elements:
    lap[0,1:-1,1:-1] = (2* umat[1, 1:-1, 1:-1] + 
                        umat[0, :-2, 1:-1] + umat[0, 2:, 1:-1] + umat[0, 1:-1, :-2] + umat[0, 1:-1, 2:] - 6*umat[0, 1:-1, 1:-1]) / (2*dx**2)
    lap[-1,1:-1,1:-1] = (2* umat[-2, 1:-1, 1:-1] + 
                        umat[-1, :-2, 1:-1] + umat[-1, 2:, 1:-1] + umat[-1, 1:-1, :-2] + umat[-1, 1:-1, 2:] - 6*umat[-1, 1:-1, 1:-1]) / (2*dx**2)
    lap[1:-1,0,1:-1] = (2* umat[1:-1, 1, 1:-1] + 
                        umat[:-2, 0, 1:-1] + umat[2:, 0, 1:-1] + umat[1:-1, 0, :-2] + umat[1:-1, 0, 2:] - 6*umat[1:-1, 0, 1:-1]) / (2*dx**2)
    lap[1:-1,-1,1:-1] = (2* umat[1:-1, -2, 1:-1] + 
                        umat[:-2, -1, 1:-1] + umat[2:, -1, 1:-1] + umat[1:-1, -1, :-2] + umat[1:-1, -1, 2:] - 6*umat[1:-1, -1, 1:-1]) / (2*dx**2)
    lap[1:-1,1:-1,0] = (2* umat[1:-1, 1:-1, 1] + 
                        umat[:-2, 1:-1, 0] + umat[2:, 1:-1, 0] + umat[1:-1, :-2, 0] + umat[1:-1, 2:, 0] - 6*umat[1:-1, 1:-1, 0]) / (2*dx**2)
    lap[1:-1,1:-1,-1] = (2* umat[1:-1, 1:-1, -2] + 
                        umat[:-2, 1:-1, -1] + umat[2:, 1:-1, -1] + umat[1:-1, :-2, -1] + umat[1:-1, 2:, -1] - 6*umat[1:-1, 1:-1, -1]) / (2*dx**2)

    # Edge Elements:
    lap[0,0,1:-1] = (2 * umat[1, 0, 1:-1] + 2 * umat[0, 1, 1:-1] + umat[0, 0, :-2] + umat[0, 0, 2:] - 6*umat[0, 0, 1:-1]) / (4*dx**2)
    lap[0,-1,1:-1] = (2 * umat[1, -1, 1:-1] + 2 * umat[0, -2, 1:-1] + umat[0, -1, :-2] + umat[0, -1, 2:] - 6*umat[0, -1, 1:-1]) / (4*dx**2)
    lap[-1,0,1:-1] = (2 * umat[2, 0, 1:-1] + 2 * umat[-1, 1, 1:-1] + umat[-1, 0, :-2] + umat[-1, 0, 2:] - 6*umat[-1, 0, 1:-1]) / (4*dx**2)
    lap[-1,-1,1:-1] = (2 * umat[2, -1, 1:-1] + 2 * umat[-1, -2, 1:-1] + umat[-1, -1, :-2] + umat[-1, -1, 2:] - 6*umat[-1, -1, 1:-1]) / (4*dx**2)
    lap[0,1:-1,0] = (2 * umat[1, 1:-1, 0] + 2 * umat[0, 1:-1, 1] + umat[0, 2:, 0] + umat[0, :-2, 0] - 6*umat[0, 1:-1, 0]) / (4*dx**2)
    lap[0,1:-1,-1] = (2 * umat[1, 1:-1, -1] + 2 * umat[0, 1:-1, -2] + umat[0, 2:, -1] + umat[0, :-2, -1] - 6*umat[0, 1:-1, -1]) / (4*dx**2)
    lap[-1,1:-1,0] = (2 * umat[-2, 1:-1, 0] + 2 * umat[-1, 1:-1, 1] + umat[-1, 2:, 0] + umat[-1, :-2, 0] - 6*umat[-1, 1:-1, 0]) / (4*dx**2)
    lap[-1,1:-1,-1] = (2 * umat[-2, 1:-1, -1] + 2 * umat[-1, 1:-1, -2] + umat[-1, 2:, -1] + umat[-1, :-2, -1] - 6*umat[-1, 1:-1, -1]) / (4*dx**2)
    lap[1:-1,0,0] = (2 * umat[1:-1, 1, 0] + 2 * umat[1:-1, 0, 1] + umat[:-2, 0, 0] + umat[2:, 0, 0] - 6*umat[1:-1, 0, 0]) / (4*dx**2)
    lap[1:-1,0,-1] = (2 * umat[1:-1, 1, -1] + 2 * umat[1:-1, 0, -2] + umat[:-2, 0, -1] + umat[2:, 0, -1] - 6*umat[1:-1, 0, -1]) / (4*dx**2)
    lap[1:-1,-1,0] = (2 * umat[1:-1, -2, 0] + 2 * umat[1:-1, -1, 1] + umat[:-2, -1, 0] + umat[2:, -1, 0] - 6*umat[1:-1, -1, 0]) / (4*dx**2)
    lap[1:-1,-1,-1] = (2 * umat[1:-1, 2, -1] + 2 * umat[1:-1, -1, -2] + umat[:-2, -1, -1] + umat[2:, -1, -1] - 6*umat[1:-1, -1, -1]) / (4*dx**2)    
    
    # Corner Elements:
    lap[0,0,0] = (umat[1, 0, 0] + umat[0, 1, 0] + umat[0, 0, 1] - 3*umat[0, 0, 0]) / (2*dx**2)
    lap[-1,0,0] = (umat[-2, 0, 0] + umat[-1, 1, 0] + umat[-1, 0, 1] - 3*umat[-1, 0, 0]) / (2*dx**2)
    lap[0,-1,0] = (umat[1, -1, 0] + umat[0, -2, 0] + umat[0, -1, 1] - 3*umat[0, -1, 0]) / (2*dx**2)
    lap[0,0,-1] = (umat[1, 0, -1] + umat[0, 1, -1] + umat[0, 0, -2] - 3*umat[0, 0, -1]) / (2*dx**2)
    lap[0,-1,-1] = (umat[1, -1, -1] + umat[0, -2, -1] + umat[0, -1, -2] - 3*umat[0, -1, -1]) / (2*dx**2)
    lap[-1,0,-1] = (umat[-2, 0, -1] + umat[-1, 1, -1] + umat[-1, 0, -2] - 3*umat[-1, 0, -1]) / (2*dx**2)
    lap[-1,-1,0] = (umat[2, -1, 0] + umat[-1, -2, 0] + umat[-1, -1, 1] - 3*umat[-1, -1, 0]) / (2*dx**2)
    lap[-1,-1,-1] = (umat[-2, -1, -1] + umat[-1, -2, -1] + umat[-1, -1, -2] - 3*umat[-1, -1, -1]) / (2*dx**2)

    return lap
  
def dudt(t, u, alpha, dx, startDay, thetaLatitude, intensity, penDepth, c, rho, Tair, heatTransferCoef, convThickness):
  dudt = alpha*lap3DFE(u,dx) + powerGen(t, umat, startDay, thetaLatitude, intensity, c, rho, penDepth) + bdryConv(t, umat, Tair, heatTransferCoef, c, rho, convThickness)
  return dudt

def dudtFlat(t, uflat, alpha, dx, startDay, thetaLatitude, intensity, penDepth, c, rho, 
          Tair, heatTransferCoef, convThickness):
    u = uflat.reshape(xmax,ymax,zmax)
    dudt3d = dudt(t, u, alpha, dx, startDay, thetaLatitude, intensity, penDepth, c, rho, 
          Tair, heatTransferCoef, convThickness)
    return dudt3d.flatten()

## Simulating a hot box
Ok, so it's time to simulate this system. First we will need to make some decisions. First, we need to decide where the system is. I will put it where I'm currently living, Greenville, NC (Latitude = $35.6 \deg$ N). Next, we need to decide what day of the year it is. Let's say it is 


Now we can finally run the model:


In [ ]:
def simToyHotBox(A,B,tmax, nt):
    oneHour = 3600
    airTemp = 27
    eqTemp = airTemp + A*solarIntensity/B * L*W/(2*(L*W + L*H + W*H))
    u0.fill(airTemp)
    time = np.arange(0,tmax,nt)
    
    hotBoxSim = solve_ivp(dudtFlat, t_span=[0,10*oneHour], y0=u0.flatten(), t_eval= time, 
                            args=[thermalDiffusivity,solarIntensity,Deltax,airTemp,A,B])

    return hotBoxSim